In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
current_directory = os.getcwd()
file_path = os.path.join(current_directory, '/content/drive/MyDrive/data/train_data.csv')
print(file_path)
finbot = pd.read_csv(file_path)

In [ ]:
finbot.head()


# Clean data for GPT2

In [ ]:
# This function creates a list of dictionaries from finbot dataframe pulling each Q and A and turinng them into a string
def prepare_data_from_csv(finbot):
    return [
        {
            "text": f"User: {row['question']}\nBot: {row['answer']}"
        }
        for _, row in finbot.iterrows()
    ]


In [ ]:
prepared_data = prepare_data_from_csv(finbot)

In [ ]:
with open("formatted_qa.txt", "w") as f:
    for item in prepared_data:
        f.write(item["text"] + "\n\n")

# Fine tuning GPT2 Model


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer


In [ ]:
pip install transformers datasets torch

In [ ]:
from datasets import load_dataset


In [ ]:
#Assigning the end of sentence token for padding and matching with our base model
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("text", data_files={"train": "formatted_qa.txt"})

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])


In [ ]:
# creating batches
from transformers import DataCollatorForLanguageModeling

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [ ]:
from transformers import TrainingArguments, Trainer


In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned-finance",
    overwrite_output_dir=True,
    num_train_epochs=4,
    per_device_train_batch_size=8,
    save_steps=1000,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=1000
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator=data_collator
)


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dimakatso-ntwampe99 (dimakatso-ntwampe99-shaper) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


In [ ]:
# saving the model and tokenizer
trainer.save_model("/content/drive/MyDrive/model/gpt2-finance-chatbot")
tokenizer.save_pretrained("/content/drive/MyDrive/model/gpt2-finance-chatbot")


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch


In [ ]:
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/model/gpt2-finance-chatbot")
tokenizer = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/model/gpt2-finance-chatbot")

In [ ]:
def generate_response(prompt):
    inputs = tokenizer.encode(prompt, return_tensors="pt")


In [ ]:
   def generate_response(prompt):
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(inputs,
        max_length=150,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
    user_input = input("Please enter your question: ")

    prompt = f"User: {user_input}\nBot:"
    response = generate_response(prompt)
    print(response.split("Bot:")[-1].strip())